In [ ]:
#Python libraries

In [ ]:
import pandas as pd
import geopandas as gpd
from shapely import wkt
from shapely.geometry import shape, box
import json
import osmnx as ox
import networkx as nx
import matplotlib.pyplot as plt
import folium
from joblib import Parallel, delayed

In [ ]:
#This Python code processes spatial data on buildings, public transport, and road networks in the city of Porto

In [ ]:
# Function to load data from CSV and convert WKT to geometry
def load_csv_with_geometry(path, geom_column='geometry_wkt'):
    df = pd.read_csv(path, dtype=str, low_memory=False)
    if geom_column in df.columns:
        df['geometry'] = df[geom_column].apply(wkt.loads)
        return gpd.GeoDataFrame(df, geometry='geometry')
    return df

# Safe function to convert GeoJSON string to shapely geometry
def safe_geojson_loads(geojson_str):
    try:
        return shape(json.loads(geojson_str))
    except (ValueError, TypeError, json.JSONDecodeError):
        return None

# Paths to relevant files
path_edificios = 'edificios.csv'
path_stcp = 'stcp.csv'
path_metro = 'metro.csv'
path_freguesias = 'freguesia.csv'

# Load data
edificios = load_csv_with_geometry(path_edificios, geom_column='geometry_wkt')
freguesias = pd.read_csv(path_freguesias, sep=';', encoding='latin1')
paragens_metro = pd.read_csv(path_metro, dtype=str, low_memory=False)
paragens_stcp = pd.read_csv(path_stcp, dtype=str, low_memory=False)

# Convert GeoJSON representations to geometric objects
freguesias['geometry'] = freguesias['Geo Shape'].apply(safe_geojson_loads)
gdf_freguesias = gpd.GeoDataFrame(freguesias, geometry='geometry').dropna(subset=['geometry'])

# Normalize parish names to handle accents
gdf_freguesias['Official Name Parish'] = gdf_freguesias['Official Name Parish'].str.normalize('NFKD').str.encode('ascii', 'ignore').str.decode('utf-8')

# Select the parish of Cedofeita, Santo Ildefonso, Sé, Miragaia, São Nicolau, and Vitória
freguesia_cssmv = gdf_freguesias[gdf_freguesias['Official Name Parish'].str.contains('Cedofeita|Santo Ildefonso|Sé|Miragaia|São Nicolau|Vitória', case=False, na=False)]

# Check if the parish was found
if freguesia_cssmv.empty:
    print("Parish of Cedofeita, Santo Ildefonso, Sé, Miragaia, São Nicolau, and Vitória not found!")
else:
    print("Parish of Cedofeita, Santo Ildefonso, Sé, Miragaia, São Nicolau, and Vitória found.")
    cssmv_geom = freguesia_cssmv.unary_union  # Merge the geometries of the parish
    
    # Convert DataFrames to GeoDataFrames if they are not already
    if not isinstance(edificios, gpd.GeoDataFrame):
        edificios = gpd.GeoDataFrame(edificios, geometry='geometry')

    # Filter data for the parish of Cedofeita, Santo Ildefonso, Sé, Miragaia, São Nicolau, and Vitória
    edificios_cssmv = edificios[edificios.geometry.within(cssmv_geom)]

    # Convert metro and STCP stops data to GeoDataFrames
    paragens_metro['geometry'] = paragens_metro.apply(lambda row: wkt.loads(f"POINT ({row['stop_lon']} {row['stop_lat']})"), axis=1)
    paragens_stcp['geometry'] = paragens_stcp.apply(lambda row: wkt.loads(f"POINT ({row['stop_lon']} {row['stop_lat']})"), axis=1)
    paragens_metro_gdf = gpd.GeoDataFrame(paragens_metro, geometry='geometry')
    paragens_stcp_gdf = gpd.GeoDataFrame(paragens_stcp, geometry='geometry')

    # Create a 1.5 km buffer around the parish geometry
    buffer_1_5km = cssmv_geom.buffer(1500)

    # Filter stops within the 1.5 km buffer
    paragens_metro_buffer = paragens_metro_gdf[paragens_metro_gdf.geometry.within(buffer_1_5km)]
    paragens_stcp_buffer = paragens_stcp_gdf[paragens_stcp_gdf.geometry.within(buffer_1_5km)]

    # Create a road network graph for the parish of Cedofeita, Santo Ildefonso, Sé, Miragaia, São Nicolau, and Vitória
    G_total = ox.graph_from_polygon(cssmv_geom, network_type='walk')

In [ ]:
#This Python script generates an interactive map using Folium and visualizes transport infrastructure 

In [ ]:
# Define the center of the map based on the geometry 
centro = [cssmv_geom.centroid.y, cssmv_geom.centroid.x]
m = folium.Map(location=centro, zoom_start=14)

# Add the road network to the map
for u, v, data in G_total.edges(data=True):
    if 'geometry' in data:
        # Add geometry if available
        folium.PolyLine([(coord[1], coord[0]) for coord in data['geometry'].coords], color="blue", weight=2.5, opacity=0.8).add_to(m)
    else:
        # If no geometry is available, draw a simple line between nodes
        x_u, y_u = G_total.nodes[u]['x'], G_total.nodes[u]['y']
        x_v, y_v = G_total.nodes[v]['x'], G_total.nodes[v]['y']
        folium.PolyLine([(y_u, x_u), (y_v, x_v)], color="blue", weight=2.5, opacity=0.8).add_to(m)

# Add metro stops to the map
for idx, row in paragens_metro.iterrows():
    folium.Marker(
        location=[row['stop_lat'], row['stop_lon']],
        popup=row['stop_name'],
        icon=folium.Icon(color='blue', icon='info-sign')
    ).add_to(m)

# Add STCP stops to the map
for idx, row in paragens_stcp.iterrows():
    folium.Marker(
        location=[row['stop_lat'], row['stop_lon']],
        popup=row['stop_name'],
        icon=folium.Icon(color='red', icon='info-sign')
    ).add_to(m)

# Add layer control
folium.LayerControl().add_to(m)

# Save the map as an HTML file
m

In [ ]:
#This Python script processes spatial data to analyze the accessibility of public transport (STCP bus stops and metro stations) for buildings in the northern part of a parish

In [ ]:
# Compute the bounding box of the area of Cedofeita, Santo Ildefonso, Sé, Miragaia, São Nicolau, and Vitória
minx, miny, maxx, maxy = cssmv_geom.bounds

# Split the area into two parts (north/south)
mid_y = (maxy + miny) / 2

# Create the geometry for the northern area
north_geom = cssmv_geom.intersection(box(minx, mid_y, maxx, maxy))

# Filter buildings and STCP and metro stops located in the northern area of Cedofeita, Santo Ildefonso, Sé, Miragaia, São Nicolau, and Vitória
buildings_north = edificios[edificios.geometry.within(north_geom)].copy()
stcp_stops_north = paragens_stcp_gdf[paragens_stcp_gdf.geometry.within(north_geom)].copy()
metro_stops_north = paragens_metro_gdf[paragens_metro_gdf.geometry.within(north_geom)].copy()

# Function to find the nearest node in the graph
def find_nearest_node(G, point):
    if point.geom_type == 'Point':
        return ox.distance.nearest_nodes(G, X=point.x, Y=point.y)
    elif point.geom_type in ['Polygon', 'MultiPolygon']:
        centroid = point.centroid
        return ox.distance.nearest_nodes(G, X=centroid.x, Y=centroid.y)
    else:
        raise ValueError(f"Unsupported geometry type: {point.geom_type}")

# Add nearest nodes for buildings and STCP and metro stops
buildings_north['nearest_node'] = buildings_north['geometry'].apply(lambda geom: find_nearest_node(G_total, geom))
stcp_stops_north['nearest_node'] = stcp_stops_north['geometry'].apply(lambda geom: find_nearest_node(G_total, geom))
metro_stops_north['nearest_node'] = metro_stops_north['geometry'].apply(lambda geom: find_nearest_node(G_total, geom))

# Function to calculate the average distance to points of interest (STCP and metro) within a 1.5 km radius
def compute_average_distance(G, building_node, stops_gdf, max_distance=1500):
    distances = []
    for stop_node in stops_gdf['nearest_node']:
        try:
            distance = nx.shortest_path_length(G, building_node, stop_node, weight='length')
            if distance <= max_distance:
                distances.append(distance)
        except nx.NetworkXNoPath:
            continue
    if distances:
        return sum(distances) / len(distances)  # Average distance
    else:
        return None  # If no stops exist within a 1.5 km radius

# Function to apply parallel computation for calculating average distances
def compute_average_distances_for_all(G, buildings_gdf, stops_gdf, max_distance=1500):
    return Parallel(n_jobs=-1)(
        delayed(compute_average_distance)(G, building_node, stops_gdf, max_distance)
        for building_node in buildings_gdf['nearest_node']
    )

# Compute average distances for all buildings in the northern area
buildings_north['average_distance_stcp_stops'] = compute_average_distances_for_all(G_total, buildings_north, stcp_stops_north, max_distance=1500)
buildings_north['average_distance_metro_stops'] = compute_average_distances_for_all(G_total, buildings_north, metro_stops_north, max_distance=1500)

# Display results for the northern area
print("Results for the Northern Area:")
print(buildings_north[['geometry', 'average_distance_stcp_stops', 'average_distance_metro_stops']])

In [ ]:
##This Python script processes spatial data to analyze the accessibility of public transport (STCP bus stops and metro stations) for buildings in the Southern part of a parish

In [ ]:
# Compute the bounding box of the area of Cedofeita, Santo Ildefonso, Sé, Miragaia, São Nicolau, and Vitória
minx, miny, maxx, maxy = cssmv_geom.bounds

# Split the area into two parts (north/south)
mid_y = (maxy + miny) / 2

# Create the geometry for the southern area
south_geom = cssmv_geom.intersection(box(minx, miny, maxx, mid_y))

# Filter buildings and STCP and metro stops located in the southern area of Cedofeita, Santo Ildefonso, Sé, Miragaia, São Nicolau, and Vitória
buildings_south = edificios[edificios.geometry.within(south_geom)].copy()
stcp_stops_south = paragens_stcp_gdf[paragens_stcp_gdf.geometry.within(south_geom)].copy()
metro_stops_south = paragens_metro_gdf[paragens_metro_gdf.geometry.within(south_geom)].copy()

# Function to find the nearest node in the graph
def find_nearest_node(G, point):
    if point.geom_type == 'Point':
        return ox.distance.nearest_nodes(G, X=point.x, Y=point.y)
    elif point.geom_type in ['Polygon', 'MultiPolygon']:
        centroid = point.centroid
        return ox.distance.nearest_nodes(G, X=centroid.x, Y=centroid.y)
    else:
        raise ValueError(f"Unsupported geometry type: {point.geom_type}")

# Add nearest nodes for buildings and STCP and metro stops
buildings_south['nearest_node'] = buildings_south['geometry'].apply(lambda geom: find_nearest_node(G_total, geom))
stcp_stops_south['nearest_node'] = stcp_stops_south['geometry'].apply(lambda geom: find_nearest_node(G_total, geom))
metro_stops_south['nearest_node'] = metro_stops_south['geometry'].apply(lambda geom: find_nearest_node(G_total, geom))

# Function to calculate the average distance to points of interest (STCP and metro) within a 1.5 km radius
def compute_average_distance(G, building_node, stops_gdf, max_distance=1500):
    distances = []
    for stop_node in stops_gdf['nearest_node']:
        try:
            distance = nx.shortest_path_length(G, building_node, stop_node, weight='length')
            if distance <= max_distance:
                distances.append(distance)
        except nx.NetworkXNoPath:
            continue
    if distances:
        return sum(distances) / len(distances)  # Average distance
    else:
        return None  # If no stops exist within a 1.5 km radius

# Function to apply parallel computation for calculating average distances
def compute_average_distances_for_all(G, buildings_gdf, stops_gdf, max_distance=1500):
    return Parallel(n_jobs=-1)(
        delayed(compute_average_distance)(G, building_node, stops_gdf, max_distance)
        for building_node in buildings_gdf['nearest_node']
    )

# Compute average distances for all buildings in the southern area
buildings_south['average_distance_stcp_stops'] = compute_average_distances_for_all(G_total, buildings_south, stcp_stops_south, max_distance=1500)
buildings_south['average_distance_metro_stops'] = compute_average_distances_for_all(G_total, buildings_south, metro_stops_south, max_distance=1500)

# Display results for the southern area
print("Results for the Southern Area:")
print(buildings_south[['geometry', 'average_distance_stcp_stops', 'average_distance_metro_stops']])

In [ ]:
#Merge Data from Northern and Southern Areas

In [ ]:
# Merge the DataFrames of the northern and southern areas
buildings_total = pd.concat([buildings_north, buildings_south], ignore_index=True)

# Display the combined results
print("Combined Results for the Northern and Southern Areas:")
print(buildings_total[['geometry', 'average_distance_stcp_stops', 'average_distance_metro_stops']])

# Save the combined DataFrame to a CSV file
output_csv_path = "file.csv"
buildings_total.to_csv(output_csv_path, index=False)

print(f"Unified CSV file saved at: {output_csv_path}")